## Python 2
Go to [Python2 C API](Python2/c_api_p2.ipynb "Python 2 C API").

In [ ]:
// hello.c file
#include <string.h>
#include <stdlib.h>

char * hello(const char * what){
  size_t what_size = strlen(what);
  char* result = (char*) malloc( (what_size + 6) * sizeof(char));
  strcpy(result, "Hello "); 
  strcat(result, what);
  return result; 
} 

In [ ]:
// hello.h header file
char * hello( const char * what); 

In [ ]:
//hellomodule.c file
#include <Python.h>
#include "hello.h"

static PyObject * hello_wrapper(PyObject * self, PyObject * args)
{
  char * input;
  char * result;
  PyObject * ret;

  // parse arguments
  if (!PyArg_ParseTuple(args, "s", &input)) {
    return NULL;
  }

  // run the actual function
  result = hello(input);

  // build the resulting string into a Python object.
  ret = PyString_FromString(result);
  free(result);

  return ret;
}

In [ ]:
//hellomodule.c file ...

static PyMethodDef HelloMethods[] = {
 { "hello", hello_wrapper, METH_VARARGS, "Say hello" },
 { NULL, NULL, 0, NULL }
};

PyMODINIT_FUNC
inithello(void)
{
     (void) Py_InitModule("hello", HelloMethods);
}

In [ ]:
#setup.py

from distutils.core import setup, Extension

# the c++ extension module
extension_mod = Extension("hello", ["hellomodule.c", "hello.c"])

setup(name = "hello", ext_modules=[extension_mod]) 
  

In [ ]:
! python2 setup.py build
! cp build/lib.linux-x86_64-2.7/hello.so .
! python2 -c "import    hello;  print(hello.hello('Asterics'))"

* PyArg_ParseTuple call. That call is what tells Python that the ‘hello’ wrapper function takes precisely one argument, a string (“s” means “string”; “ss” would mean “two strings”; “si” would mean “string and integer”). 

* The convention in the C API to Python is that a NULL return from a function that returns PyObject* indicates an error has occurred; in this case, the error information is set within PyArg_ParseTuple and we’re just passing the error on up the stack by returning NULL.

* references. Python works on a system of reference counting: each time a function “takes ownership” of an object (by, for example, assigning it to a list, or a dictionary) it increments that object’s reference count by one using Py_INCREF.

* When the object is removed from use in that particular place (e.g. removed from the list or dictionary), the reference count is decremented with Py_DECREF.
 
* When the reference count reaches 0, Python knows that this object is not being used by anything and can be freed (it may not be freed immediately, however).

* Why does this matter? Well, we’re creating a PyObject in this code, with PyString_FromString. Do we need to INCREF it? To find out, go take a look at the documentation for PyString_FromString:

    https://docs.python.org/2/c-api/string.html

See where it says “New reference”? That means it’s handing back an object with a reference count of 1, and that’s what we want. 

If it had said “Borrowed reference”, then we would need to INCREF the object before returning it, to indicate that we wanted the allocated memory to survive past the end of the function.

* if you receive a Python object from the Python API, you can use it within your own C code without INCREFing it.
* if you want to guarantee that the Python object survives past the end of your own C code, you must INCREF it.
* if you received an object from Python code and it was a new reference, but you don’t want it to survive past the end of your own C code, you should DECREF it.

* If you wanted to return None, by the way, you can use Py_None. Remember to INCREF it!